In [ ]:
from itertools import permutations
import heapq
import pandas as pd
import numpy as np
from functools import reduce


class MDijkstra:
    def __init__(self,adj_matrix,start,pretuplist):
        self.adj_matrix = adj_matrix
        self.start = start
        #prelist格式[(1,2),(1,1),...],优先级生序
        self.prelist = pretuplist

    def MatrixExpansion(self):
        # 获取邻接矩阵的行数和列数
        rows, cols = self.adj_matrix.shape
        # 初始化一个与self.adjmatrix 相同大小的矩阵 matmin
        matmin = np.zeros_like(self.adj_matrix)
        # 遍历行和列
        for i in range(rows):
            for j in range(cols):
                mat = self.adj_matrix[i,j]
                # 对每个块矩阵元素求最小值
                if np.all(mat == 0):
                    matmin[i, j] = 0
                else:
                    non_zero_elements = mat[mat != 0]
                    matmin[i, j] = np.min(non_zero_elements)
        non_zero_element_min = np.min(matmin[matmin != 0])
        max_decimal_places = -np.inf
        decimal_places = len(str(non_zero_element_min).split(".")[1]) if "." in str(non_zero_element_min) else 0
        max_decimal_places = max(max_decimal_places, decimal_places)
        scaling_factor = 10 ** max_decimal_places
        adj_matrix_adjust = scaling_factor * self.adj_matrix
        return adj_matrix_adjust

    def MatrixAdjust(self):
        taget_raw_mat = self.MatrixExpansion()
        taget_list = self.prelist
        multi_list = []
        # 获取邻接矩阵的行数和列数
        rows, cols = taget_raw_mat.shape
        # 初始化一个与 taget_raw_mat 相同大小的全1矩阵 matsum
        matsum = np.ones_like(taget_raw_mat)
        # 遍历行和列
        for i in range(rows):
            for j in range(cols):
                # 对每个块矩阵元素求和
                matsum[i, j] = np.sum(taget_raw_mat[i, j])
        print(matsum)

        for ele in taget_list:
            multi_list.append(matsum[ele[0], ele[1]])
        print(multi_list)

        def eleone(ele):
            if ele == 0:
                return 1
            else:
                return ele

        multi_list = [eleone(ele) for ele in multi_list]
        print(multi_list)
        multi_final_list = [1]
        for k in range(1,len(multi_list)):
            product = reduce((lambda x, y: x * y), multi_list[:k])
            multi_final_list.append(product)
        print(multi_final_list)

        for num in range(len(taget_list)):
            row = taget_list[num][0]
            col = taget_list[num][1]
            taget_raw_mat[row,col] = taget_raw_mat[row,col].astype(np.float64)
            taget_raw_mat[row,col] *= multi_final_list[num]

        return np.vstack([np.hstack(taget_raw_mat[i]) for i in range(len(taget_raw_mat))])

    def M_dijkstra(self):
        adj_matrix = self.MatrixAdjust()
        num_vertices = len(adj_matrix)
        start = self.start
        # 初始化距离列表和前驱节点列表
        distances = [float('infinity')] * num_vertices
        distances[start] = 0
        predecessors = [None] * num_vertices

        # 使用优先队列（最小堆）存储（距离，节点）对
        priority_queue = [(0, start)]

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)

            # 如果当前距离比已记录的距离长，则忽略
            if current_distance > distances[current_vertex]:
                continue

            # 更新邻居节点的距离和前驱节点
            for neighbor in range(num_vertices):
                weight = adj_matrix[current_vertex][neighbor]
                if weight > 0:  # 考虑权重大于0的边
                    distance = current_distance + weight
                    if distance < distances[neighbor]:
                        distances[neighbor] = distance
                        predecessors[neighbor] = current_vertex
                        heapq.heappush(priority_queue, (distance, neighbor))

        return (distances, predecessors)

    def print_shortest_path(self):
        predecessors = self.M_dijkstra()[1]
        distances = self.M_dijkstra()[0]
        start = self.start
        endlist = list(range(len(self.MatrixAdjust())))
        endlist.remove(start)
        df = pd.DataFrame(columns=['start', 'end', 'path', 'length'])
        for end in endlist:
            path = []
            current = end
            while current is not None:
                path.insert(0, current)
                current = predecessors[current]
            new_row = {'start': start, 'end': end, 'path': path, 'length':distances[end]}
            new_row_df = pd.DataFrame([new_row])
            df = pd.concat([df, new_row_df], ignore_index=True)
        return df


if __name__ == '__main__':
    def is_zero_matrix(matrix):
        for row in matrix:
            for element in row:
                if element != 0:
                    return False
        return True

    # 目标序列
    sequence = [(0,0),(1,1),(2,2),(3,3),(0,1),(1,2),(2,3)]
    presequence = [(0,2),(0,3),(1,3),(1,0),(2,0),(2,1),(3,0),(3,1),(3,2)]

    # 获取序列的所有排列
    all_permutations = list(permutations(sequence))
    target_list = []
    # 所有排列值
    for perm in all_permutations:
        data = presequence+list(perm)
        target_list.append(data)

    # 读取数据
    block_matrix_seq = np.load('block_matrix_seq.npy',allow_pickle=True)

    print(len(target_list))

    # datares = MDijkstra(block_matrix_seq[6], 0, target_list[6])
    # print(datares.print_shortest_path())

    minlist = []
    for ele in target_list[0:4]:
        df = pd.DataFrame(columns=['start', 'end', 'path', 'length'])
        for num in range(28):
            datares = MDijkstra(block_matrix_seq[1],num,ele)
            dataadj = datares.print_shortest_path()
            df = pd.concat([df,dataadj], ignore_index=True)
        df = df[(df['start']<28) & (df['end']>=63) & (df['length'] != np.inf)]
        avg = df['length'].mean()
        minlist.append(avg)

    print(minlist)
    min_value = min(minlist)
    min_index = minlist.index(min_value)

    df_final = pd.DataFrame(columns=['start', 'end', 'path', 'length'])
    for num in range(28):
        datares_final = MDijkstra(block_matrix_seq[1], num, target_list[min_index])
        dataadj_final = datares_final.print_shortest_path()
        df_final = pd.concat([df_final,dataadj_final], ignore_index=True)
    df_final = df_final[(df_final['start'] < 28) & (df_final['end'] >= 63) & (df_final['length'] != np.inf)]

    writer = pd.ExcelWriter(str(target_list[min_index][9:16])+'.xlsx')
    df_final.to_excel(writer)
    writer.close()


